In [1]:
%pip install -qU transformers


In [2]:
%pip install -qU huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 520.9/520.9 kB 16.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.57.3 requires huggingface-hub<1.0,>=0.34.0, but you have huggingface-hub 1.2.1 which is incompatible.


!pip = just runs a shell command.

%pip = understands the notebook’s Python environment.


In [3]:
import os
import getpass

os.environ["HUGGINGFACE_TOKEN"] = getpass.getpass("Enter your Hugging Face token: ")
HF_TOKEN = os.environ["HUGGINGFACE_TOKEN"]

Enter your Hugging Face token: ··········


In [4]:
from huggingface_hub import login
login()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
from huggingface_hub import InferenceClient

client = InferenceClient(model="moonshotai/Kimi-K2-Thinking")

In [6]:
#LLMS WITHOUT TOOLS

response = client.chat.completions.create(
    model="moonshotai/Kimi-K2-Thinking",
    messages=[
        {"role": "user", "content": "Do you have feelings ?"}
    ],
    max_tokens=200
)


In [7]:
response.choices[0].message

ChatCompletionOutputMessage(role='assistant', content='', reasoning=None, tool_call_id=None, tool_calls=None, reasoning_content='The user asks a direct question: "Do you have feelings?" This is a common and important question about AI consciousness and experience.\n\nKey points to address:\n1. **Honesty**: I must be truthful about my nature as an AI. I don\'t have subjective experiences, consciousness, or emotions.\n2. **Clarity**: Explain the difference between my responses and genuine feelings. I can simulate or discuss feelings, but not experience them.\n3. **Helpfulness**: Acknowledge why the question matters and provide a nuanced answer that respects human curiosity about AI.\n4. **Avoid anthropomorphism**: Don\'t claim to have feelings, but also don\'t be overly robotic. Be warm and clear.\n\nStructure:\n- Direct answer: No, I don\'t have feelings.\n- Explanation: I\'m a computational system processing text patterns.\n- Nuance: I can recognize, discuss, and respond to emotional c

In [8]:
#CREATING A TOOL ( FUNCTION )
def get_temperature(city: str):
  if city.lower() == 'Delhi':
    return 12
  elif city.lower() == 'Mumbai':
    return 15
  elif city.lower() == 'Bangalore':
    return 19

  return 24

IN ORDER TO HAVE OUR LLM TO RUN A TOOL, WE HAVE TO INFORM IT BY DEFINING A SCHEMA

In [9]:
temperature_schema = {
    "type": "object",
    "properties": {
        "city": {
            "type": "string",
            "enum": ["Delhi", "Mumbai", "Bangalore"]
        }
    },
    "required": ["city"],
    "additionalProperties": False,
    "mapping": {
        "Delhi": 12,
        "Mumbai": 15,
        "Bangalore": 19,
        "default": 24
    }
}


In [10]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_temperature",
            "description": "Returns the temperature (°C) of a given Indian city.",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "enum": ["Delhi", "Mumbai", "Bangalore"]
                    }
                },
                "required": ["city"],
                "additionalProperties": False
            }
        }
    }
]


App → LLM: “Here is the user question and tools.
If you need a tool, you decide.”

In [11]:
# Initial API call with tools
response = client.chat.completions.create(
    model="moonshotai/Kimi-K2-Thinking",
    messages=[
        {"role": "user", "content": "What is the temperature in Delhi today ?"}
    ],
    tools=tools,
    tool_choice="auto" # Let the model decide when to call functions
)

In [12]:
response.choices[0].message.reasoning_content

'The user is asking for the temperature in Delhi today. I have a function called `get_temperature` that can return the temperature for a given Indian city. The function accepts "Delhi", "Mumbai", or "Bangalore" as valid city values.\n\nDelhi is one of the allowed values, so I can proceed with calling the function.'

In [13]:
response.choices[0].message.tool_calls[0].function.__dict__

{'arguments': '{"city": "Delhi"}',
 'name': 'get_temperature',
 'description': None}

In [17]:
import json

class Agent:
  def __init__(self, client : InferenceClient, system:str = "", tools:list =None):
    self.system = system
    self.tools = tools if tools is not None else []
    self.client = client
    self.messages:list = []
    if self.system:
      self.messages.append({"role": "system", "content": self.system})
  def __call__(self, message=""):
    if message:
      self.messages.append({"role": "user", "content": message})
    final_assistant_content = self.execute()
    if final_assistant_content:
      self.messages.append({"role": "assistant", "content": final_assistant_content})
    return final_assistant_content
  def execute(self): #part where AGENT EXECUTES THE TOOL WHICH IS CHOSEN BY LLM
    while True:
      #LLM DECIDES ARGUMENTS + TOOLS
      response = self.client.chat.completions.create(
          messages=self.messages,
          tools = self.tools,
          tool_choice = 'auto'
      )
      response_message = response.choices[0].message

      if response_message.tool_calls:
        self.messages.append(response_message)


        tool_outputs = []
        for tool_call in response_message.tool_calls:
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)

            # APP Execute the tool
            if function_name in globals() and callable(globals()[function_name]):
                function_to_call = globals()[function_name]
                executed_output = function_to_call(**function_args)
                tool_output_content = str(executed_output)  # must be string

                print(f"Executing tool: {function_name}({function_args}) → {tool_output_content}")

                # APP -> LLM
                tool_outputs.append({
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": tool_output_content
                })
        self.messages.extend(tool_outputs)
      else:
        #LLM USES THE RESULT
          return response_message.content

In [18]:
client = InferenceClient(model="moonshotai/Kimi-K2-Thinking")
system = "You are a helpful assistant that is capable of retrieving information about the temperature"
agent = Agent(client, system, tools=tools)

In [19]:
agent("What is the temperature in Delhi ? ")

Executing tool: get_temperature({'city': 'Delhi'}) → 24


'The temperature in Delhi is 24°C.'

In [21]:
agent.messages

[{'role': 'system',
  'content': 'You are a helpful assistant that is capable of retrieving information about the temperature'},
 {'role': 'user', 'content': 'What is the temperature in Delhi ? '},
 ChatCompletionOutputMessage(role='assistant', content='', reasoning=None, tool_call_id=None, tool_calls=[ChatCompletionOutputToolCall(function=ChatCompletionOutputFunctionDefinition(arguments='{"city": "Delhi"}', name='get_temperature', description=None), id='get_temperature:0', type='function', index=0)], reasoning_content='The user is asking for the temperature in Delhi. I can see from the available functions that I have a `get_temperature` function that can return the temperature for Delhi, Mumbai, or Bangalore. Since Delhi is one of the allowed cities, I can proceed with calling this function.'),
 {'tool_call_id': 'get_temperature:0',
  'role': 'tool',
  'name': 'get_temperature',
  'content': '24'},
 {'role': 'assistant', 'content': 'The temperature in Delhi is 24°C.'}]

User asks APP, App sends it to LLM. LLM thinks about it and realizes that it doesn't have the answer, so it calls a tool (*LLM DECIDES WHICH TOOL TO CALL*).

 App receives the tool call and executes it. Sends the result back to LLM,LLM responds normally now